# Stability, order and accuracy for non-stiff and stiff equations

In [ ]:
import numpy as np
from scipy.optimize import root
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

## Curtiss and Hirschfelder

We consider the following problem :

$$
\left\{ 
\begin{aligned} 
{\mathrm d}_t u(t) & = k \, \big(cos(t) - u(t)) \big) \quad \text{with } k > 1\\ 
u(t_0)  & = u_0 
\end{aligned} 
\right. 
$$

and in the following, we will assume $t_0=0$.

### Stiffness

The exact solution is given by :

$$
u(t) = \frac{k}{k^2+1} \bigg( k \cos(t) + \sin(t) \bigg) + c_0 \, e^{-k t} \quad
\text{avec} \quad c_0 = \bigg( u_0 -\frac{k}{k^2 + 1} \Big( k \cos(t_0) + \sin(t_0) \Big) \bigg)  e^{-k t_0}   
$$

In [ ]:
class curtiss_model:

    def __init__(self, k):
        self.k = k

    def fcn(self, t, u) :
        k = self.k
        u_dot = k * (np.cos(t) - u)
        return u_dot

    def sol(self, uini, t0, t):
        k = self.k

        c0 = (uini - (k/(k*k + 1)) * (k*np.cos(t0) + np.sin(t0))) * np.exp(k*t0)
        u = (k/(k*k + 1)) * (k*np.cos(t) + np.sin(t)) +  c0 * np.exp(-k*t)
        return u

In [ ]:
uini = 2.
tini = 0.
tend = 10.
k = 10.

cm = curtiss_model(k)
    
texa = np.linspace(tini, tend, 1000)
uexa = cm.sol(uini, tini, texa)

fig = go.Figure()
fig.add_trace(go.Scatter(x=texa, y=uexa, name='f(x)'))

#create slider
steps = []
for k_i in range(10, 101, 10):
    cm = curtiss_model(k_i)
    step = dict(method="update", label = f"{k_i}", args=[{"x": [texa], "y": [cm.sol(uini, tini, texa)]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'k = '}, steps=steps)]

fig.update_layout(sliders=sliders, title = 'Exact solution of the Curtiss and Hirschfelder equation', 
                  xaxis_title="t", yaxis_title="u")
fig['layout']['sliders'][0]['pad']=dict(t= 50)
fig.show()

## Explicit Euler

The explicit Euler method to solve ${\mathrm d}_t u(t) = f(t,u)$ with $u(0)=u_0$ can be written :

$$
\left\{
\begin{aligned}
& U^0 = u_0 \\
& U^{n+1} = U^n + \Delta t \; f(t^n,U^n) \quad \text{where} \quad \Delta t = t^{n+1} - t^n
\end{aligned}
\right.
$$

In [ ]:
#####################################################
class ode_result:
    def __init__(self, y, t):
        self.y = y
        self.t = t

#####################################################
def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt))
    y[:,0] = yini

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*np.array(fcn(tn, yn))

    return ode_result(y, t)

### Solution and error

In [ ]:
def sol_and_error_curtiss(yini, tini, tend, k, nt, method):
    
    if method == forward_euler:
        str_method = "Explicit euler"
    elif method == backward_euler:
        str_method = "Implicit euler"
    else:
        print("Unknown method")
        return
    
    cm = curtiss_model(k)
    fcn = cm.fcn
    
    texa = np.linspace(tini, tend, 500)
    yexa = cm.sol(yini, tini, texa)

    nt_max= nt[-1]
    dt_max = (tend-tini)/(nt_max-1)
    sol = method(tini, tend, nt_max, yini, fcn)
    err = np.abs(sol.y[0]-cm.sol(yini, tini, sol.t))[1:]
    
    # first plot
    fig = make_subplots(rows=2, cols=1, vertical_spacing=0.15)
    color = fig.layout.template.layout.colorway
    fig.add_trace(go.Scatter(x=texa, y=yexa, name='exact sol', line_color='grey', legendgroup = 'sol'), 
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=sol.t, y=sol.y[0], mode='markers+lines', line_dash='dot', #marker=marker01, 
                             line_color=color[0], name='Sol. num', legendgroup = 'sol'), row=1, col=1)
    fig.add_trace(go.Scatter(x=sol.t[1:], y=err, mode='markers+lines', line_dash='dot', #marker=marker02, 
                             line_color=color[1], name='erreur', legendgroup = 'err'), row=2, col=1)
    
    # create slider
    steps = []
    for i, nt_i in enumerate(nt):
        sol = method(tini, tend, nt_i, yini, fcn)
        err = np.abs(sol.y[0]-cm.sol(yini, tini, sol.t))[1:]
        dt = (tend-tini)/(nt_i-1)
        step = dict(method="update", label = f"{nt_i}", args=[{"x": [texa, sol.t, sol.t[1:]], 
                                                               "y": [yexa, sol.y[0], err]},
                {"title": f"Solution and error for k={int(k)}, dt={dt:.4e} and k.dt={k*dt:.2f}"}])
        steps.append(step)

    sliders = [dict(active=len(nt)-1, currentvalue={"prefix": "nt : "}, steps=steps)]

    fig.update_xaxes(title='t')
    fig.update_yaxes(row=1, col=1, title='u')
    fig.update_yaxes(row=2, col=1, exponentformat='e', title='|error|')
    legend = dict(tracegroupgap=300, groupclick="toggleitem", x=0.8, bgcolor='rgba(0,0,0,0)')
    fig.update_layout(height=800, sliders=sliders, legend=legend, 
                      title=str_method+ f" : solution and error for k={int(k)}, dt={dt_max:.3e} and k.dt={k*dt:.2f}")
    fig['layout']['sliders'][0]['pad']=dict(t= 50)
    fig.show()

In [ ]:
yini = 2.
tini = 0.
tend = 2.
k = 50.

nt_max = 200
nt = np.hstack((np.arange(k-5, k+2, 1, dtype=int) , np.geomspace(k+2, nt_max, num=12, dtype=int)))

sol_and_error_curtiss(yini, tini, tend, k, nt, forward_euler)

### Order

In [ ]:
uini = 2.
tini = 0.
tend = 2.
cm = curtiss_model(k=100)
fcn = cm.fcn

nt = np.array([201, 2001, 20001, 200001])

err_1   = np.zeros_like(nt, dtype=float)
err_2   = np.zeros_like(nt, dtype=float)
err_inf = np.zeros_like(nt, dtype=float)

for i, nt_i in enumerate(nt):
    sol = forward_euler(tini, tend, nt_i, uini, fcn)
    uerr = np.abs(cm.sol(uini, tini, sol.t) - sol.y[0])
    err_1[i]   = np.linalg.norm(uerr,1) / nt_i
    err_2[i]   = np.linalg.norm(uerr) / np.sqrt(nt_i)   
    err_inf[i] = np.linalg.norm(uerr,np.inf) 

dt = (tend-tini)/(nt-1)

fig = go.Figure()
fig.add_trace(go.Scatter(x=dt, y=err_1, name='Norm l1'))
fig.add_trace(go.Scatter(x=dt, y=err_2, name='Norm l2'))
fig.add_trace(go.Scatter(x=dt, y=err_inf, name='Norm linf'))
fig.add_trace(go.Scatter(x=dt, y=dt, mode='lines', line_dash='dot', name='slope 1'))
fig.update_xaxes(type='log', exponentformat='e', title='dt')
fig.update_yaxes(type='log', exponentformat='e', title='error norm')
fig.show()

## Backward Euler

The backward Euler method to solve ${\mathrm d}_t u(t) = f(t,u)$ with $u(0) = u_0$ can be written :

\begin{equation*}
\left\{
\begin{aligned}
& U^0 = u_0 \\
& U^{n+1} = U^n + \Delta t \; f(t^{n+1},U^{n+1}), \qquad \Delta t = t^{n+1} - t^n,
\end{aligned}
\right.
\end{equation*}

In [ ]:
#####################################################
def backward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt))
    y[:,0] = yini

    def g(uip1, *args):
        uip, tip1 = args
        return uip1 - uip - dt*np.array(fcn(tip1, uip1))

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y0 = yn + dt*np.array(fcn(tn, yn))
        # solve y[:,it+1] - y[:,it] - dt * fcn(tini + (it+1)*dt, y[:,it+1]) = 0
        sol = root(g, y0, (yn, tn+dt))
        y[:,it+1] = sol.x

    return ode_result(y, t)

### Solution and error

In [ ]:
yini = 2.
tini = 0.
tend = 2.
k = 60.

nt_max = 200
nt = np.linspace(10, nt_max, 20, dtype=int)

sol_and_error_curtiss(yini, tini, tend, k, nt, backward_euler)

### Order

In [ ]:
uini = 2.
tini = 0.
tend = 2.
cm = curtiss_model(k=100)
fcn = cm.fcn

nt = np.array([201, 2001, 20001, 200001])

err_1   = np.zeros_like(nt, dtype=float)
err_2   = np.zeros_like(nt, dtype=float)
err_inf = np.zeros_like(nt, dtype=float)

for i, nt_i in enumerate(nt):
    sol = backward_euler(tini, tend, nt_i, uini, fcn)
    uerr = np.abs(cm.sol(uini, tini, sol.t) - sol.y[0])
    err_1[i]   = np.linalg.norm(uerr,1) / nt_i
    err_2[i]   = np.linalg.norm(uerr) / np.sqrt(nt_i)   
    err_inf[i] = np.linalg.norm(uerr,np.inf) 

dt = (tend-tini)/(nt-1)

fig = go.Figure()
fig.add_trace(go.Scatter(x=dt, y=err_1, name='Norm l1'))
fig.add_trace(go.Scatter(x=dt, y=err_2, name='Norm l2'))
fig.add_trace(go.Scatter(x=dt, y=err_inf, name='Norm linf'))
fig.add_trace(go.Scatter(x=dt, y=dt, mode='lines', line_dash='dot', name='slope 1'))
fig.update_xaxes(type='log', exponentformat='e', title='dt')
fig.update_yaxes(type='log', exponentformat='e', title='error norm')
fig.show()